# Creation of the CNN

### Importations 

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl


from torch.autograd import Variable
from torch import nn, optim
from torch.nn import functional as F
from matplotlib import gridspec




print(torch.__version__)

1.0.1.post2


### The Aim 

Some explation about the purpose of this project.

### The Data

Some insight about the data.

In [69]:
path = "/Users/youssef/Documents/EPFL/DataSet/Training_Set/"
i = 0
data_matrix = np.zeros([4,4,129,390])
for i in range(4):
    name_matrix = 'phase_matrix/Phase_matrix-{}'.format(i)
    fileName_matrix = path + name_matrix
    fileObject2 = open(fileName_matrix, 'rb')
    matrix_loaded = pkl.load(fileObject2)
    fileObject2.close()
    data_matrix[i] = matrix_loaded
data_matrix.shape    

(4, 4, 129, 390)

In [70]:
data_matrix = torch.from_numpy(data_matrix)
data_matrix = data_matrix.view(-1,390,4,129)
data_matrix = data_matrix.type('torch.FloatTensor')
type(data_matrix)

torch.Tensor

In [120]:
def prep_targets(target,index,label):
    target = torch.zeros(1, dtype=torch.float64)
    target[index] = label
    target = target.expand(390,1)
    return target

In [245]:
indices = torch.tensor([9, 9, 3, 1])
targets = torch.zeros([4,390,1], dtype=torch.float32)
for i in range(4):
    targets[i] = prep_targets(targets,i,indices[i])

targets.shape

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [132]:
target_0 = torch.tensor(9)
target_0 = target_0.expand(390,1)
target_1 = torch.tensor(9)
target_1 = target_1.expand(390,1)
target_2 = torch.tensor(3)
target_2 = target_2.expand(390,1)
target_3 = torch.tensor(1)
target_3 = target_3.expand(390,1)
targets = torch.zeros([4,390,1], dtype=torch.float32)
targets[0] = target_0
targets[1] = target_1
targets[2] = target_2
targets[3] = target_3
targets.shape

torch.Size([4, 390, 1])

# Architecture 1.0 : vanilla CNN for DOA 

In this section, we will focus on building a vanilla CNN in order to recognize the directions of arrival of the sound in a specific room. Once done, we will then compare "by hand" the labelel dataset to the predicted values. This is the most basic setup and will try to improve latter.

## 1.1 Creating Model

In [239]:
class Net(nn.Module):
    def __init__(self,nb_hidden = 512):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv3 = nn.Conv2d(64 , 64, kernel_size=2)
        self.fc1 = nn.Linear(8064, nb_hidden)# (1x8064) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.fc2 = nn.Linear(nb_hidden, 37)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 8064)))
        x = F.relu(self.fc2(x))
        print(x)
        return F.softmax(x)

### Preparing the Data

In [187]:
def prep_input_vanilla(train_input):
    new_train_input = train_input.view(-1,390,1,1,4,129)
    return new_train_input

In [203]:
train_data = prep_input_vanilla(data_matrix)
train_data

tensor([[[[[[ 0.0000,  0.0000,  0.0000,  ...,  3.1416,  3.1416,  3.1416],
            [ 3.1416,  3.1416,  3.1416,  ...,  0.0000,  0.0000,  3.1416],
            [ 0.0000,  3.1416,  0.0000,  ...,  3.1416,  3.1416,  3.1416],
            [ 3.1416,  3.1416,  0.0000,  ...,  0.4785, -0.8493,  0.5713]]]],



         [[[[-1.3277,  0.9593, -0.3950,  ..., -0.2487,  0.9203, -1.0584],
            [ 1.8859, -2.9608,  0.2918,  ...,  1.3497, -1.6369,  2.8038],
            [-0.7857, -0.4618, -0.1938,  ..., -1.3317,  2.0755, -1.8986],
            [ 2.1879,  2.6223, -2.0051,  ..., -0.3469, -0.0955, -1.2927]]]],



         [[[[-0.2508, -0.5966,  2.0186,  ...,  1.4449,  0.4941,  0.1378],
            [-1.9248,  1.8002,  0.1563,  ..., -2.8957, -2.1876,  0.7372],
            [-2.6450,  0.6864,  1.0434,  ...,  2.0940, -2.4613,  1.0596],
            [ 2.1687, -1.3951,  1.9594,  ...,  2.5936,  2.1695, -1.4983]]]],



         ...,



         [[[[ 0.2208, -0.2802, -2.5366,  ...,  2.3999, -2.0372, -2.9199],
   

In [189]:
def prep_labels_vanilla(train_input):
    new_train_input = train_input.view(-1,390,1)
    return new_train_input

In [190]:
train_targets = prep_labels_vanilla(targets)
train_targets.shape

torch.Size([4, 390, 1])

In [191]:
train_targets = train_targets.to(dtype=torch.int64)

In [192]:
train_data, train_targets = Variable(train_data), Variable(train_targets)

In [193]:
train_targets

tensor([[[9],
         [9],
         [9],
         ...,
         [9],
         [9],
         [9]],

        [[9],
         [9],
         [9],
         ...,
         [9],
         [9],
         [9]],

        [[3],
         [3],
         [3],
         ...,
         [3],
         [3],
         [3]],

        [[1],
         [1],
         [1],
         ...,
         [1],
         [1],
         [1]]])

### End of Sanity check:

## 1.2 Training the model

In [241]:
mini_batch_size = 30 
nb_epochs = 1
eta = 0.001 #learning rate
#criterion = nn.MSELoss() # MeanSquaredloss
criterion = torch.nn.CrossEntropyLoss() #Cross Entropy

In [242]:
def train_model(model, train_input, train_target, mini_batch_size, criterion, nb_epochs, eta):
    
    optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent
    
    for e in range(0, nb_epochs):
        # We do this with mini-batches
        for b in range(0, train_input.size(0)):
            for a in range(0,train_input.size(1)):
                output = model(train_input[b,a,:,:,:,:])
                target = train_target[b,a,:]
                print("Output vs target",output,target)
                #loss = criterion(output,target)
                loss = criterion(output,target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
        print(e, loss.data.item())
    

In [180]:
def train_model(model, train_input, train_target, mini_batch_size, criterion, nb_epochs, eta):
    
    optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent
    
    for e in range(0, nb_epochs):
        # We do this with mini-batches
        for b in range(0, train_input.size(0)):
            for a in range(0,train_input.size(1),mini_batch_size):
                output = model(train_input[b].narrow(0, a, mini_batch_size))
                #print(output.shape)
                target = train_targets[b].narrow(0, a, mini_batch_size)
                #print(target.shape)
                #loss = criterion(output,target)
                loss = criterion(output,targets.max(1)[1])
                print(b,target.max(1)[1])
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        print(e, loss.data.item())

In [243]:
model = Net()

In [244]:
train_model(model,train_data,train_targets,mini_batch_size,criterion,nb_epochs,eta)

tensor([[0.0173, 0.0000, 0.0000, 0.0000, 0.0032, 0.0000, 0.0000, 0.0000, 0.0143,
         0.0757, 0.0646, 0.0000, 0.0345, 0.0108, 0.0000, 0.0169, 0.0702, 0.0301,
         0.0000, 0.0000, 0.0368, 0.0115, 0.0313, 0.0270, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0522, 0.0044,
         0.0754]], grad_fn=<ReluBackward0>)
Output vs target tensor([[0.0271, 0.0266, 0.0266, 0.0266, 0.0267, 0.0266, 0.0266, 0.0266, 0.0270,
         0.0287, 0.0284, 0.0266, 0.0275, 0.0269, 0.0266, 0.0271, 0.0285, 0.0274,
         0.0266, 0.0266, 0.0276, 0.0269, 0.0274, 0.0273, 0.0266, 0.0266, 0.0266,
         0.0266, 0.0266, 0.0266, 0.0266, 0.0266, 0.0266, 0.0266, 0.0280, 0.0267,
         0.0287]], grad_fn=<SoftmaxBackward>) tensor([9])
tensor([[0.0171, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0293, 0.0528, 0.0000, 0.0350, 0.0228, 0.0000, 0.0178, 0.0577, 0.0193,
         0.0000, 0.0000, 0.0163, 0.0079, 0.0000, 0.0534, 0.0183, 0.0000

/Users/youssef/env/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.0217, 0.0137, 0.0040, 0.0000, 0.0000, 0.0000, 0.0000, 0.0112, 0.0000,
         0.0417, 0.0819, 0.0199, 0.0680, 0.0275, 0.0264, 0.0311, 0.0669, 0.0247,
         0.0000, 0.0000, 0.0194, 0.0000, 0.0234, 0.0426, 0.0144, 0.0067, 0.0000,
         0.0000, 0.0028, 0.0000, 0.0000, 0.0482, 0.0000, 0.0000, 0.0081, 0.0000,
         0.0600]], grad_fn=<ReluBackward0>)
Output vs target tensor([[0.0271, 0.0269, 0.0266, 0.0265, 0.0265, 0.0265, 0.0265, 0.0268, 0.0265,
         0.0277, 0.0288, 0.0271, 0.0284, 0.0273, 0.0272, 0.0274, 0.0284, 0.0272,
         0.0265, 0.0265, 0.0271, 0.0265, 0.0272, 0.0277, 0.0269, 0.0267, 0.0265,
         0.0265, 0.0266, 0.0265, 0.0265, 0.0279, 0.0265, 0.0265, 0.0268, 0.0265,
         0.0282]], grad_fn=<SoftmaxBackward>) tensor([9])
tensor([[0.0053, 0.0098, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0177, 0.0054,
         0.0458, 0.0664, 0.0055, 0.0475, 0.0311, 0.0022, 0.0475, 0.0557, 0.0386,
         0.0000, 0.0059, 0.0243, 0.0000, 0.0126, 0.0416, 0.0273, 0.0391

tensor([[0.0090, 0.0038, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0132,
         0.0994, 0.0275, 0.0000, 0.0616, 0.0370, 0.0000, 0.0121, 0.0566, 0.0236,
         0.0000, 0.0257, 0.0066, 0.0000, 0.0053, 0.0313, 0.0000, 0.0230, 0.0000,
         0.0000, 0.0000, 0.0087, 0.0000, 0.0067, 0.0000, 0.0000, 0.0000, 0.0131,
         0.0527]], grad_fn=<ReluBackward0>)
Output vs target tensor([[0.0269, 0.0267, 0.0266, 0.0266, 0.0266, 0.0266, 0.0266, 0.0266, 0.0270,
         0.0294, 0.0274, 0.0266, 0.0283, 0.0277, 0.0266, 0.0270, 0.0282, 0.0273,
         0.0266, 0.0273, 0.0268, 0.0266, 0.0268, 0.0275, 0.0266, 0.0273, 0.0266,
         0.0266, 0.0266, 0.0269, 0.0266, 0.0268, 0.0266, 0.0266, 0.0266, 0.0270,
         0.0281]], grad_fn=<SoftmaxBackward>) tensor([9])
tensor([[0.0188, 0.0025, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1044, 0.0467, 0.0326, 0.0657, 0.0176, 0.0000, 0.0465, 0.0855, 0.0110,
         0.0000, 0.0000, 0.0252, 0.0000, 0.0000, 0.0338, 0.0195, 0.0093

tensor([[0.0000, 0.0014, 0.0045, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0118,
         0.1718, 0.0537, 0.0000, 0.0476, 0.0183, 0.0000, 0.0225, 0.0588, 0.0218,
         0.0000, 0.0068, 0.0047, 0.0000, 0.0183, 0.0468, 0.0073, 0.0239, 0.0000,
         0.0000, 0.0069, 0.0000, 0.0000, 0.0083, 0.0000, 0.0000, 0.0280, 0.0033,
         0.0301]], grad_fn=<ReluBackward0>)
Output vs target tensor([[0.0266, 0.0266, 0.0267, 0.0266, 0.0266, 0.0266, 0.0266, 0.0266, 0.0269,
         0.0316, 0.0280, 0.0266, 0.0279, 0.0271, 0.0266, 0.0272, 0.0282, 0.0272,
         0.0266, 0.0268, 0.0267, 0.0266, 0.0271, 0.0279, 0.0268, 0.0272, 0.0266,
         0.0266, 0.0268, 0.0266, 0.0266, 0.0268, 0.0266, 0.0266, 0.0273, 0.0267,
         0.0274]], grad_fn=<SoftmaxBackward>) tensor([9])
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0011, 0.0000, 0.0000, 0.0148, 0.0080,
         0.1828, 0.0349, 0.0000, 0.0261, 0.0258, 0.0000, 0.0459, 0.0663, 0.0186,
         0.0000, 0.0163, 0.0063, 0.0000, 0.0175, 0.0546, 0.0053, 0.0015

...

Output vs target tensor([[2.0538e-26, 3.0331e-26, 2.0538e-26, 2.9583e-26, 2.0538e-26, 2.0538e-26,
         2.1787e-26, 2.0538e-26, 2.0792e-26, 1.0000e+00, 2.0538e-26, 2.2431e-26,
         2.0538e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26, 3.6452e-26, 2.0538e-26,
         2.7117e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26,
         2.2604e-26, 2.2563e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26, 2.0753e-26,
         2.0538e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26, 2.0538e-26,
         2.8804e-26]], grad_fn=<SoftmaxBackward>) tensor([1])
tensor([[0.0000e+00, 4.3897e-01, 0.0000e+00, 4.1286e-01, 0.0000e+00, 0.0000e+00,
         3.4229e-02, 0.0000e+00, 1.6528e-02, 5.8250e+01, 0.0000e+00, 1.0114e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.5453e-01, 0.0000e+00,
         2.6933e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         9.3134e-02, 1.2581e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.1207e-03,
         0.0000e+00, 0.0000e+0

tensor([[0.0000e+00, 3.2459e-01, 0.0000e+00, 3.6010e-01, 0.0000e+00, 0.0000e+00,
         5.5156e-02, 0.0000e+00, 4.0037e-02, 5.7335e+01, 0.0000e+00, 1.0169e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.5286e-01, 0.0000e+00,
         3.4298e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         8.4266e-02, 1.2872e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.3480e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         3.8128e-01]], grad_fn=<ReluBackward0>)
Output vs target tensor([[1.2581e-25, 1.7406e-25, 1.2581e-25, 1.8035e-25, 1.2581e-25, 1.2581e-25,
         1.3295e-25, 1.2581e-25, 1.3095e-25, 1.0000e+00, 1.2581e-25, 1.3928e-25,
         1.2581e-25, 1.2581e-25, 1.2581e-25, 1.2581e-25, 2.1869e-25, 1.2581e-25,
         1.7729e-25, 1.2581e-25, 1.2581e-25, 1.2581e-25, 1.2581e-25, 1.2581e-25,
         1.3687e-25, 1.4310e-25, 1.2581e-25, 1.2581e-25, 1.2581e-25, 1.3010e-25,
         1.2581e-25, 1.2581e-25, 1.2581e-25,

tensor([[0.0000e+00, 3.6380e-01, 0.0000e+00, 4.0670e-01, 0.0000e+00, 0.0000e+00,
         1.2051e-01, 0.0000e+00, 1.8973e-02, 5.5689e+01, 0.0000e+00, 1.5147e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.5156e-01, 0.0000e+00,
         2.3093e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.4442e-01, 1.4419e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7480e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         3.6256e-01]], grad_fn=<ReluBackward0>)
Output vs target tensor([[6.5261e-25, 9.3896e-25, 6.5261e-25, 9.8012e-25, 6.5261e-25, 6.5261e-25,
         7.3619e-25, 6.5261e-25, 6.6511e-25, 1.0000e+00, 6.5261e-25, 7.5933e-25,
         6.5261e-25, 6.5261e-25, 6.5261e-25, 6.5261e-25, 1.2521e-24, 6.5261e-25,
         8.2214e-25, 6.5261e-25, 6.5261e-25, 6.5261e-25, 6.5261e-25, 6.5261e-25,
         7.5400e-25, 7.5383e-25, 6.5261e-25, 6.5261e-25, 6.5261e-25, 6.6411e-25,
         6.5261e-25, 6.5261e-25, 6.5261e-25,

tensor([[0.0000e+00, 4.2267e-01, 0.0000e+00, 3.8508e-01, 0.0000e+00, 0.0000e+00,
         5.8431e-02, 0.0000e+00, 4.8167e-02, 5.9315e+01, 0.0000e+00, 8.7487e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.7876e-01, 0.0000e+00,
         3.2341e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0370e-01, 1.6453e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.0144e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         4.2578e-01]], grad_fn=<ReluBackward0>)
Output vs target tensor([[1.7377e-26, 2.6518e-26, 1.7377e-26, 2.5540e-26, 1.7377e-26, 1.7377e-26,
         1.8423e-26, 1.7377e-26, 1.8235e-26, 1.0000e+00, 1.7377e-26, 1.8966e-26,
         1.7377e-26, 1.7377e-26, 1.7377e-26, 1.7377e-26, 3.4258e-26, 1.7377e-26,
         2.4013e-26, 1.7377e-26, 1.7377e-26, 1.7377e-26, 1.7377e-26, 1.7377e-26,
         1.9276e-26, 2.0485e-26, 1.7377e-26, 1.7377e-26, 1.7377e-26, 1.8089e-26,
         1.7377e-26, 1.7377e-26, 1.7377e-26,

tensor([[0.0000e+00, 3.8740e-01, 0.0000e+00, 3.8572e-01, 0.0000e+00, 0.0000e+00,
         1.0112e-01, 0.0000e+00, 3.3608e-02, 5.8095e+01, 0.0000e+00, 3.7186e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.2545e-01, 0.0000e+00,
         2.2387e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.4833e-02, 9.2938e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         3.9117e-01]], grad_fn=<ReluBackward0>)
Output vs target tensor([[5.8862e-26, 8.6713e-26, 5.8862e-26, 8.6567e-26, 5.8862e-26, 5.8862e-26,
         6.5126e-26, 5.8862e-26, 6.0874e-26, 1.0000e+00, 5.8862e-26, 6.1092e-26,
         5.8862e-26, 5.8862e-26, 5.8862e-26, 5.8862e-26, 1.1002e-25, 5.8862e-26,
         7.3632e-26, 5.8862e-26, 5.8862e-26, 5.8862e-26, 5.8862e-26, 5.8862e-26,
         6.2805e-26, 6.4595e-26, 5.8862e-26, 5.8862e-26, 5.8862e-26, 5.8862e-26,
         5.8862e-26, 5.8862e-26, 5.8862e-26,